In [28]:
# Use SelectKBest for feature selection
# k_best_features = 30  
# select_k_best = SelectKBest(f_regression, k=k_best_features)
# X_selected = select_k_best.fit_transform(X, y)
# Use forward selection for feature selection
# k_best_features = 30
# forward_selector = SequentialFeatureSelector(LinearRegression(), k_features=k_best_features, forward=True, scoring='neg_mean_squared_error', cv=5)
# X_selected = forward_selector.fit_transform(X, y)
# Use statsmodels to perform OLS regression and get p-values




In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from mlxtend.feature_selection import SequentialFeatureSelector
import csv

# Load the training data
df = pd.read_csv('train.csv')

# Handle 'sub_area' column
category = df['sub_area'].value_counts()
others_col = category[category < 1000].index.tolist()
df['sub_area'] = df['sub_area'].replace(others_col, 'others')

# Perform dummy encoding
df = pd.get_dummies(df, drop_first=True)

# Separate features (X) and target variable (y)
X = df.drop(['price_doc'], axis=1)
y = df['price_doc']

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:


# Use forward selection for feature selection
k_best_features = 25  # Set the number of features to select
forward_selector = SequentialFeatureSelector(LinearRegression(), k_features=k_best_features, forward=True, scoring='neg_mean_squared_error', cv=5)
X_selected = forward_selector.fit_transform(X, y)

# Use PolynomialFeatures to create polynomial features
degree = 2
poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X_selected)

# Fit a linear regression model on the polynomial features
reg = LinearRegression().fit(X_poly, y)





In [ ]:
# Load the test data
df_test = pd.read_csv('test.csv')

# Drop unnecessary column from the test set
df_test = df_test.drop(['row ID'], axis=1)

# Handle 'sub_area' column in the test set
df_test['sub_area'] = df_test['sub_area'].replace(others_col, 'others')

# Perform dummy encoding for the test set
df_test = pd.get_dummies(df_test, drop_first=True)

# Standardize the test set features
df_test = scaler.transform(df_test)

# Extract the selected features for the test set using forward selection
X_test_selected = forward_selector.transform(df_test)

# Use PolynomialFeatures to create polynomial features for the test set
df_test_poly = poly.transform(X_test_selected)


In [ ]:
# Predict on the test set using the polynomial features
y_pred_test = reg.predict(df_test_poly)

# Save the predictions to a CSV file
filepath = 'prediction_forward_selection_poly.csv'
with open(filepath, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['row ID', 'price_doc'])  # Add column headers
    for c, i in enumerate(y_pred_test, start=1):
        writer.writerow([c, i])